### Imports and initializations

In [1]:
# Import the necessary libraries
import pandas as pd
import time
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', 20)

# Call the stats endpoints that I need for this.
from nba_api.stats.endpoints import leaguedashplayerstats
from nba_api.stats.endpoints import leaguedashplayerptshot
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.static import players

In [2]:
# This is just to see a sample of the dataframe for the player dashboard
# test = leaguedashplayerstats.LeagueDashPlayerStats(season_type_all_star='Regular Season',\
#                                                    measure_type_detailed_defense='Base',\
#                                                    # per_mode_detailed='Per100Possessions')
# test.get_data_frames()[0]

In [3]:
# Create a list of the seasons from 1996-97 (as far as the API goes back to) until this year
# Make sure it's output is in a YYYY-YY format.
seasons = [f'{year}-{str(year + 1)[-2:]}' for year in range(1996, 2024)]
print(seasons)

['1996-97', '1997-98', '1998-99', '1999-00', '2000-01', '2001-02', '2002-03', '2003-04', '2004-05', '2005-06', '2006-07', '2007-08', '2008-09', '2009-10', '2010-11', '2011-12', '2012-13', '2013-14', '2014-15', '2015-16', '2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22', '2022-23', '2023-24']


In [4]:
# len(seasons)

---
### Grabbing seasonal player statistics from the NBA API
Just grabbing the raw data (and tagging the mode and season on there).

In [5]:
# This is to grab the player dashboard for a specific year (with any particular measures and modes)
# Return the dataframe
def get_player_season_dashboard(season, measure, mode):
    current_season = leaguedashplayerstats.LeagueDashPlayerStats(season_type_all_star='Regular Season',\
                                                   measure_type_detailed_defense=measure,\
                                                   per_mode_detailed=mode,\
                                                                 season=season)
    current_season_df = current_season.get_data_frames()[0]
    current_season_df['season'] = season  # Must tag the set as the season before sending it back out
    return current_season_df

def get_player_shooting_dashboard(season, measure='Totals'):
    current_season = leaguedashplayerptshot.LeagueDashPlayerPtShot(season_type_all_star='Regular Season',\
                                               per_mode_simple=measure,\
                                               season=season)
    current_season_df = current_season.get_data_frames()[0]
    current_season_df['season'] = season  # Must tag the set as the season before sending it back out
    return current_season_df

# Use this to concatenate the new dataframe(df2) to the original one (df1)
def add_to_current_dataframe(df1, df2, index=True):
    return pd.concat([df1, df2], ignore_index=index)

# Take in the list of seasons and grab all the dashboards for each season in the list
# Defaults to the basic stats and per 100 possessions, but can change it to Advanced and/or Per36
# def get_dashboard_all_seasons(seasons=['2023-24'], measure='Base', permode='Per100Possesions'):
def get_dashboard_all_seasons(seasons, measure, permode, label):
    seasons_df = pd.DataFrame()
    for season in seasons:
        current_season = get_player_season_dashboard(season, measure, permode)
        current_season['mode'] = label # Must tag it as the per_x mode before sending it back out
        seasons_df = add_to_current_dataframe(seasons_df, current_season)
    return seasons_df

# This gets the shooting data (for the 2PT% data)
def get_shooting_all_seasons(seasons, measure):
    seasons_df = pd.DataFrame()
    for season in seasons:
        current_season = get_player_shooting_dashboard(season, measure)
        seasons_df = add_to_current_dataframe(seasons_df, current_season)
    # seasons_df = seasons_df.rename(columns={'PLAYER_LAST_TEAM_ID': 'team_id', 'PLAYER_LAST_TEAM_ABBREVIATION': 'team_abbreviation'})
    return seasons_df

In [6]:
# # Uncomment if you need to pull the data from the API

# # This is where we grab all the basic and advanced stats per 36 minutes and per 100 possesions and store them
# # into dataframes
# per_36_min_basic_stats = get_dashboard_all_seasons(seasons, 'Base', 'Per36', '36_min')
# # per_36_min_adv_stats = get_dashboard_all_seasons(seasons, 'Advanced', 'Per36', '36_min')
# per_100_poss_basic_stats = get_dashboard_all_seasons(seasons, 'Base', 'Per100Possessions', '100_poss')
# # per_100_poss_adv_stats = get_dashboard_all_seasons(seasons, 'Advanced', 'Per100Possessions', '100_poss')
# totals_basic_stats = get_dashboard_all_seasons(seasons, 'Base', 'Totals', 'total')
# totals_adv_stats = get_dashboard_all_seasons(seasons, 'Advanced', 'Totals', 'total') # Grab this one because
#                                                                                        # apparently there's no difference in the modes

# totals_shooting_stats = get_shooting_all_seasons(seasons, 'Totals') 

# # Rather than doing this every time, store them in CSVs just in case.
# per_36_min_basic_stats.to_csv('per_36_min_basic_stats_raw.csv', index=False)
# # per_36_min_adv_stats.to_csv('per_36_min_adv_stats_raw.csv', index=False)
# per_100_poss_basic_stats.to_csv('per_100_poss_basic_stats_raw.csv', index=False)
# # per_100_poss_adv_stats.to_csv('per_100_poss_adv_stats_raw.csv', index=False)
# totals_basic_stats.to_csv('totals_basic_stats_raw.csv', index=False)
# totals_adv_stats.to_csv('totals_adv_stats_raw.csv', index=False)
# totals_shooting_stats.to_csv('player_shooting_stats_raw.csv', index=False)

---
### Grabbing player common info data
Just grabbing the raw common player info for each player. Note: This takes a while so might want to do it and do other things
in the meantime. Can take 60+ minutes!

In [7]:
# Grab the common info for a player, based on the player ID sent in
def get_player_common_info(id):
    return commonplayerinfo.CommonPlayerInfo(player_id=id).get_data_frames()[0]

In [8]:
# # Run this if you haven't gotten the player_common_info.csv yet!

# # This is where we grab the common player info for each player that ever played since 1996-1997.
# common_info_df = pd.DataFrame()

# for id in per_36_min_basic_stats['PLAYER_ID'].unique():
#     common_info = get_player_common_info(id)
#     common_info_df = add_to_current_dataframe(common_info_df, common_info)
#     time.sleep(0.65) # Make sure to have a sleep so that the API doesn't kick us for too many requests.

# common_info_df.to_csv('common_player_info_raw.csv', index=False)

---
### Using the data from the CSVs instead
Once you have them saved on CSVs, it's probably better to pull the data from there than from
the API itself each time.

In [9]:
# Use this cell instead to import the CSVs that were created in the previous cell if you don't want to
# run through the APIs every time you run this (assuming you have the CSVs)
per_36_min_basic_stats = pd.read_csv('per_36_min_basic_stats_raw.csv')
per_36_min_adv_stats = pd.read_csv('per_36_min_adv_stats_raw.csv')
per_100_poss_basic_stats = pd.read_csv('per_100_poss_basic_stats_raw.csv')
per_100_poss_adv_stats = pd.read_csv('per_100_poss_adv_stats_raw.csv')
totals_basic_stats = pd.read_csv('totals_basic_stats_raw.csv')
totals_adv_stats = pd.read_csv('totals_adv_stats_raw.csv')
totals_shooting_stats = pd.read_csv('player_shooting_stats_raw.csv')

In [10]:
# # Run this if you already have the CSV. Also, makes sure that the person_id gets changed to player_id.
common_info_df = pd.read_csv('common_player_info_raw.csv')

In [11]:
# Let's just to make sure the dataframe is loaded
common_info_df.head()

,PERSON_ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FI_LAST,PLAYER_SLUG,BIRTHDATE,SCHOOL,COUNTRY,LAST_AFFILIATION,HEIGHT,WEIGHT,SEASON_EXP,JERSEY,POSITION,ROSTERSTATUS,GAMES_PLAYED_CURRENT_SEASON_FLAG,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CODE,TEAM_CITY,PLAYERCODE,FROM_YEAR,TO_YEAR,DLEAGUE_FLAG,NBA_FLAG,GAMES_PLAYED_FLAG,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,GREATEST_75_FLAG
0,920,A.C.,Green,A.C. Green,"Green, A.C.",A. Green,ac-green,1963-10-04T00:00:00,Oregon State,USA,Oregon State/USA,6-9,225.0,16,45.0,Forward,Inactive,N,1610612747,Lakers,LAL,lakers,Los Angeles,a.c._green,1985,2000,N,Y,Y,1985,1,23,N
1,243,Aaron,McKie,Aaron McKie,"McKie, Aaron",A. McKie,aaron-mckie,1972-10-02T00:00:00,Temple,USA,Temple/USA,6-5,209.0,14,2.0,Guard,Inactive,N,1610612755,76ers,PHI,sixers,Philadelphia,aaron_mckie,1994,2006,N,Y,Y,1994,1,17,N
2,1425,Aaron,Williams,Aaron Williams,"Williams, Aaron",A. Williams,aaron-williams,1971-10-02T00:00:00,Xavier,USA,Xavier/USA,6-9,235.0,14,NaN,Center-Forward,Inactive,N,0,NaN,NaN,NaN,NaN,aaron_williams,1993,2007,N,Y,Y,Undrafted,Undrafted,Undrafted,N
3,768,Acie,Earl,Acie Earl,"Earl, Acie",A. Earl,acie-earl,1970-06-23T00:00:00,Iowa,USA,Iowa/USA,6-11,240.0,4,55.0,Forward-Center,Inactive,N,1610612738,Celtics,BOS,celtics,Boston,acie_earl,1993,1996,N,Y,Y,1993,1,19,N
4,228,Adam,Keefe,Adam Keefe,"Keefe, Adam",A. Keefe,adam-keefe,1970-02-22T00:00:00,Stanford,USA,Stanford/USA,6-9,230.0,9,32.0,Forward,Inactive,N,1610612762,Jazz,UTA,jazz,Utah,adam_keefe,1992,2000,N,Y,Y,1992,1,10,N


---
### Data Cleaning
This is where we start data cleaning

In [12]:
# # Using this cell to check to see if there's any duplicate names and if they're assigned properly to different player IDs
# duplicated_names = common_info_df.duplicated(subset=['first_name', 'last_name'], keep=False)
# common_info_df[duplicated_names].sort_values('display_first_last')

In [13]:
# Data cleaning functions
def columns_to_lower(df):
    df.columns = [column.lower() for column in df.columns]
    return df

def rename_column(df, original, new):
    return df.rename(columns={original: new})

def set_type(df, column, type):
    return df.astype({column: type})

**Create an inches column to help with creating a player type based on height.**

Wanted to use height and categorize them into bigs, mids, and smalls based on that because
players are much different now that they can do so many things. There's no traditional center now
and no traditional point guard or shooting guard.

In [14]:
# This sets the inches for each player so we can use it to categorize players
def set_inches(row):
    try:
        height = row['height'].split('-')
        return float(height[0]) * 12 + float(height[1])
    except:
        return 0.0

# Take the inches and categorize a player into either bid, mid, or small. Can't use 
# regular positions now because of so many positionless players + players that can do things
# (like big men shooting from range or getting a lot of assists)
def set_player_type(row):
    inches = row['inches']
    if inches >= 82:
        return 'big'
    elif inches >= 76:
        return 'mid'
    elif inches == 0:
        return 'na'
    else:
        return 'small'

In [15]:
common_info_df = columns_to_lower(common_info_df)
common_info_df['inches'] = common_info_df.apply(set_inches, axis=1)
common_info_df['player_type'] = common_info_df.apply(set_player_type, axis=1)

In [16]:
common_stats = ['player_id', 'player_name', 'height', 'inches', 'player_type']
common_info_df = rename_column(common_info_df, 'person_id', 'player_id')
common_info_df = rename_column(common_info_df, 'display_first_last', 'player_name')
common_info = common_info_df[common_stats]
common_info.shape

(2638, 5)

In [17]:
common_info = common_info[common_info['player_type'] != 'na']

In [18]:
common_info.to_csv('common_player_info.csv', index=False)

**Making all the columns lowercase here**

Just makes it easier than having ALL CAPS EVERY SINGLE TIME

In [19]:
per_36_min_basic_stats = columns_to_lower(per_36_min_basic_stats)
per_36_min_adv_stats = columns_to_lower(per_36_min_adv_stats)
per_100_poss_basic_stats = columns_to_lower(per_100_poss_basic_stats)
per_100_poss_adv_stats = columns_to_lower(per_100_poss_adv_stats)
totals_basic_stats = columns_to_lower(totals_basic_stats)
totals_adv_stats = columns_to_lower(totals_adv_stats)
totals_shooting_stats = columns_to_lower(totals_shooting_stats)

In [20]:
per_36_min_basic_stats.head()

,player_id,player_name,nickname,team_id,team_abbreviation,age,gp,w,l,w_pct,min,fgm,fga,fg_pct,fg3m,fg3a,fg3_pct,ftm,fta,ft_pct,oreb,dreb,reb,ast,tov,stl,blk,blka,pf,pfd,pts,plus_minus,nba_fantasy_pts,dd2,td3,wnba_fantasy_pts,gp_rank,w_rank,l_rank,w_pct_rank,min_rank,fgm_rank,fga_rank,fg_pct_rank,fg3m_rank,fg3a_rank,fg3_pct_rank,ftm_rank,fta_rank,ft_pct_rank,oreb_rank,dreb_rank,reb_rank,ast_rank,tov_rank,stl_rank,blk_rank,blka_rank,pf_rank,pfd_rank,pts_rank,plus_minus_rank,nba_fantasy_pts_rank,dd2_rank,td3_rank,wnba_fantasy_pts_rank,season,mode
0,920,A.C. Green,A.C.,1610612742,DAL,33.0,83,23,60,0.277,2494.298333,3.4,7.0,0.483,0.0,0.3,0.050,1.8,2.8,0.650,3.2,6.3,9.5,1.0,1.1,1.0,0.2,0.6,2.1,0.1,8.6,-5.1,24.1,16,0,21.6,1,238,13,354,77,366,396,101,298,279,302,297,271,318,88,62,72,370,412,250,323,252,404,54,380,325,341,49,24,372,1996-97,36_min
1,243,Aaron McKie,Aaron,1610612765,DET,24.0,83,48,35,0.578,1623.911667,3.3,8.1,0.411,0.9,2.3,0.398,2.0,2.4,0.836,0.9,4.0,4.9,3.6,2.0,1.7,0.5,0.8,2.9,0.0,9.6,2.9,25.4,1,0,23.4,1,67,132,158,168,370,365,302,177,200,63,261,314,60,335,211,254,124,271,71,224,177,317,192,346,90,297,168,24,324,1996-97,36_min
2,1425,Aaron Williams,Aaron,1610612763,VAN,25.0,33,4,29,0.121,562.423333,5.4,9.5,0.574,0.0,0.1,0.000,2.1,3.1,0.673,4.0,5.2,9.2,1.0,2.0,1.0,1.9,0.4,4.6,0.0,13.0,-6.7,32.0,0,0,28.9,329,392,177,425,301,137,294,8,303,338,303,252,235,296,36,127,84,376,255,245,43,343,119,192,201,372,102,219,24,138,1996-97,36_min
3,768,Acie Earl,Acie,1610612749,MIL,27.0,47,14,33,0.298,500.141667,4.8,12.9,0.374,0.0,0.4,0.000,3.9,6.0,0.643,2.5,4.4,6.9,1.4,2.5,1.1,2.0,0.6,4.4,0.0,13.5,-4.3,30.7,0,0,28.1,287,308,146,342,311,204,130,381,303,274,303,64,43,324,154,179,177,321,139,226,35,275,132,192,179,304,135,219,24,162,1996-97,36_min
4,228,Adam Keefe,Adam,1610612762,UTA,27.0,62,48,14,0.774,916.788333,3.2,6.3,0.513,0.0,0.0,0.000,2.8,4.0,0.689,2.9,5.5,8.5,1.3,1.8,1.2,0.5,0.5,3.8,0.0,9.2,4.8,24.6,1,0,22.3,215,67,320,39,252,381,416,52,303,352,303,152,135,278,116,108,110,344,323,197,210,308,206,89,362,52,323,168,24,354,1996-97,36_min


**Grabbing the columns that are of interest in the advanced stats/metrics**

Just going to be a pain to have to deal with all those rankings. Best not to.

In [21]:
adv_stats = ['player_id', 'player_name', 'team_id', 'team_abbreviation', 'season', 'mode', 'w_pct', 'min', 'off_rating', 'def_rating', 'net_rating',\
             'ast_to', 'ast_ratio', 'oreb_pct', 'dreb_pct', 'reb_pct','e_tov_pct', 'efg_pct', 'ts_pct', 'usg_pct', 'pie']
totals_adv_stats[adv_stats].head(5)

,player_id,player_name,team_id,team_abbreviation,season,mode,w_pct,min,off_rating,def_rating,net_rating,ast_to,ast_ratio,oreb_pct,dreb_pct,reb_pct,e_tov_pct,efg_pct,ts_pct,usg_pct,pie
0,920,A.C. Green,1610612742,DAL,1996-97,total,0.277,30.1,97.4,104.8,-7.4,0.93,9.6,0.089,0.184,0.135,10.4,0.485,0.523,0.118,0.098
1,243,Aaron McKie,1610612765,DET,1996-97,total,0.578,19.6,101.5,97.8,3.7,1.79,24.2,0.026,0.113,0.071,13.5,0.467,0.524,0.142,0.095
2,1425,Aaron Williams,1610612763,VAN,1996-97,total,0.121,17.0,93.3,102.6,-9.3,0.47,7.0,0.113,0.144,0.129,14.8,0.574,0.599,0.161,0.112
3,768,Acie Earl,1610612749,MIL,1996-97,total,0.298,10.6,93.6,100.0,-6.4,0.57,7.3,0.067,0.122,0.094,12.9,0.374,0.435,0.220,0.056
4,228,Adam Keefe,1610612762,UTA,1996-97,total,0.774,14.8,106.6,99.4,7.2,0.71,11.3,0.096,0.158,0.129,15.9,0.513,0.572,0.124,0.083


In [22]:
totals_adv = totals_adv_stats[adv_stats]

*Note: I forgot that all the advanced stats are the same because they're normalized across a season, so no point in having them over multiple 
modes. So will just be using the totals in the future.*

**Grabbing the columns of interest in the basic stats/metrics**

Same deal as with the advanced metrics. Just grab what we need.

In [23]:
per_36_min_basic_stats.head()

,player_id,player_name,nickname,team_id,team_abbreviation,age,gp,w,l,w_pct,min,fgm,fga,fg_pct,fg3m,fg3a,fg3_pct,ftm,fta,ft_pct,oreb,dreb,reb,ast,tov,stl,blk,blka,pf,pfd,pts,plus_minus,nba_fantasy_pts,dd2,td3,wnba_fantasy_pts,gp_rank,w_rank,l_rank,w_pct_rank,min_rank,fgm_rank,fga_rank,fg_pct_rank,fg3m_rank,fg3a_rank,fg3_pct_rank,ftm_rank,fta_rank,ft_pct_rank,oreb_rank,dreb_rank,reb_rank,ast_rank,tov_rank,stl_rank,blk_rank,blka_rank,pf_rank,pfd_rank,pts_rank,plus_minus_rank,nba_fantasy_pts_rank,dd2_rank,td3_rank,wnba_fantasy_pts_rank,season,mode
0,920,A.C. Green,A.C.,1610612742,DAL,33.0,83,23,60,0.277,2494.298333,3.4,7.0,0.483,0.0,0.3,0.050,1.8,2.8,0.650,3.2,6.3,9.5,1.0,1.1,1.0,0.2,0.6,2.1,0.1,8.6,-5.1,24.1,16,0,21.6,1,238,13,354,77,366,396,101,298,279,302,297,271,318,88,62,72,370,412,250,323,252,404,54,380,325,341,49,24,372,1996-97,36_min
1,243,Aaron McKie,Aaron,1610612765,DET,24.0,83,48,35,0.578,1623.911667,3.3,8.1,0.411,0.9,2.3,0.398,2.0,2.4,0.836,0.9,4.0,4.9,3.6,2.0,1.7,0.5,0.8,2.9,0.0,9.6,2.9,25.4,1,0,23.4,1,67,132,158,168,370,365,302,177,200,63,261,314,60,335,211,254,124,271,71,224,177,317,192,346,90,297,168,24,324,1996-97,36_min
2,1425,Aaron Williams,Aaron,1610612763,VAN,25.0,33,4,29,0.121,562.423333,5.4,9.5,0.574,0.0,0.1,0.000,2.1,3.1,0.673,4.0,5.2,9.2,1.0,2.0,1.0,1.9,0.4,4.6,0.0,13.0,-6.7,32.0,0,0,28.9,329,392,177,425,301,137,294,8,303,338,303,252,235,296,36,127,84,376,255,245,43,343,119,192,201,372,102,219,24,138,1996-97,36_min
3,768,Acie Earl,Acie,1610612749,MIL,27.0,47,14,33,0.298,500.141667,4.8,12.9,0.374,0.0,0.4,0.000,3.9,6.0,0.643,2.5,4.4,6.9,1.4,2.5,1.1,2.0,0.6,4.4,0.0,13.5,-4.3,30.7,0,0,28.1,287,308,146,342,311,204,130,381,303,274,303,64,43,324,154,179,177,321,139,226,35,275,132,192,179,304,135,219,24,162,1996-97,36_min
4,228,Adam Keefe,Adam,1610612762,UTA,27.0,62,48,14,0.774,916.788333,3.2,6.3,0.513,0.0,0.0,0.000,2.8,4.0,0.689,2.9,5.5,8.5,1.3,1.8,1.2,0.5,0.5,3.8,0.0,9.2,4.8,24.6,1,0,22.3,215,67,320,39,252,381,416,52,303,352,303,152,135,278,116,108,110,344,323,197,210,308,206,89,362,52,323,168,24,354,1996-97,36_min


In [24]:
basic_stats = ['player_id', 'player_name', 'team_id', 'team_abbreviation', 'age', 'season', 'mode', 'gp', 'min', 'w_pct', 'fgm', 'fga', 'fg_pct',\
                        'fg3m', 'fg3a', 'fg3_pct', 'ftm', 'fta', 'ft_pct', 'oreb', 'dreb', 'reb', 'ast', 'tov', 'stl', 'blk',\
                       'pts', 'plus_minus']
# per_100_poss_basic_stats[basic_stats]
per_36_basic = per_36_min_basic_stats[basic_stats]
per_100_basic = per_100_poss_basic_stats[basic_stats]
totals_basic = totals_basic_stats[basic_stats]

In [25]:
# Have to rename the two columns...
totals_shooting_stats = totals_shooting_stats.rename(columns={'player_last_team_id': 'team_id', 'player_last_team_abbreviation': 'team_abbreviation'})
# totals_shooting_stats
shooting_stats = ['player_id', 'player_name', 'team_id', 'team_abbreviation', 'season', 'fg2a_frequency', 'fg2a', 'fg3a_frequency', 'fg3a', 'fg2_pct']
totals_shooting = totals_shooting_stats[shooting_stats].copy()
totals_shooting

,player_id,player_name,team_id,team_abbreviation,season,fg2a_frequency,fg2a,fg3a_frequency,fg3a,fg2_pct
0,201142,Kevin Durant,1610612760,OKC,2013-14,0.704,1159,0.296,487,0.567
1,2546,Carmelo Anthony,1610612752,NYK,2013-14,0.737,1156,0.263,413,0.498
2,200746,LaMarcus Aldridge,1610612757,POR,2013-14,0.989,1367,0.011,15,0.475
3,201567,Kevin Love,1610612750,MIN,2013-14,0.633,857,0.367,497,0.537
4,201942,DeMar DeRozan,1610612761,TOR,2013-14,0.846,1130,0.154,206,0.474
...,...,...,...,...,...,...,...,...,...,...
5711,1631243,Mouhamed Gueye,1610612737,ATL,2023-24,0.000,0,1.000,1,NaN
5712,1631120,JD Davison,1610612738,BOS,2023-24,0.000,0,1.000,1,NaN
5713,1631376,Dmytro Skapintsev,1610612752,NYK,2023-24,1.000,1,0.000,0,0.000
5714,1641806,Markquis Nowell,1610612761,TOR,2023-24,1.000,1,0.000,0,0.000


In [26]:
totals_shooting[totals_shooting.isnull().any(axis=1)]

,player_id,player_name,team_id,team_abbreviation,season,fg2a_frequency,fg2a,fg3a_frequency,fg3a,fg2_pct
462,2586,Keith Bogans,1610612738,BOS,2013-14,0.0,0,1.0,5,NaN
957,203958,Andre Dawkins,1610612748,MIA,2014-15,0.0,0,1.0,6,NaN
963,203479,Jamaal Franklin,1610612743,DEN,2014-15,0.0,0,1.0,2,NaN
1439,203925,Joe Harris,1610612739,CLE,2015-16,0.0,0,1.0,4,NaN
1443,101126,Nate Robinson,1610612740,NOP,2015-16,0.0,0,1.0,1,NaN
...,...,...,...,...,...,...,...,...,...,...
5708,1641809,Drew Peterson,1610612738,BOS,2023-24,0.0,0,1.0,1,NaN
5709,1641775,Jordan Walsh,1610612738,BOS,2023-24,0.0,0,1.0,1,NaN
5711,1631243,Mouhamed Gueye,1610612737,ATL,2023-24,0.0,0,1.0,1,NaN
5712,1631120,JD Davison,1610612738,BOS,2023-24,0.0,0,1.0,1,NaN


In [27]:
# Have had some null values in the 2013-14 data, so going to fill them with 0% (never take a shot is still 0% here)
totals_shooting['fg2_pct'].fillna(0, inplace=True)

In [28]:
totals_shooting[totals_shooting['player_name'] == 'Terry Rozier']

,player_id,player_name,team_id,team_abbreviation,season,fg2a_frequency,fg2a,fg3a_frequency,fg3a,fg2_pct
1341,1626179,Terry Rozier,1610612738,BOS,2015-16,0.730,73,0.270,27,0.301
1657,1626179,Terry Rozier,1610612738,BOS,2016-17,0.540,210,0.460,179,0.448
2011,1626179,Terry Rozier,1610612738,BOS,2017-18,0.478,355,0.522,388,0.451
2584,1626179,Terry Rozier,1610612738,BOS,2018-19,0.493,328,0.507,337,0.424
3030,1626179,Terry Rozier,1610612766,CHA,2019-20,0.546,507,0.454,421,0.440
3531,1626179,Terry Rozier,1610612766,CHA,2020-21,0.496,562,0.504,570,0.512
4086,1626179,Terry Rozier,1610612766,CHA,2021-22,0.491,567,0.509,587,0.520
4680,1626179,Terry Rozier,1610612766,CHA,2022-23,0.577,680,0.423,499,0.482
5243,1626179,Terry Rozier,1610612748,MIA,2023-24,0.582,339,0.418,243,0.522


### Joining the tables together

Let's start joining the tables together.

In [29]:
per_100_basic.head(5)

,player_id,player_name,team_id,team_abbreviation,age,season,mode,gp,min,w_pct,fgm,fga,fg_pct,fg3m,fg3a,fg3_pct,ftm,fta,ft_pct,oreb,dreb,reb,ast,tov,stl,blk,pts,plus_minus
0,920,A.C. Green,1610612742,DAL,33.0,1996-97,100_poss,83,53.1,0.277,5.0,10.3,0.483,0.0,0.4,0.050,2.7,4.2,0.650,4.7,9.2,14.0,1.5,1.6,1.5,0.3,12.7,-7.4
1,243,Aaron McKie,1610612765,DET,24.0,1996-97,100_poss,83,52.7,0.578,4.9,11.8,0.411,1.3,3.3,0.398,3.0,3.6,0.836,1.3,5.9,7.2,5.2,2.9,2.5,0.7,14.0,4.3
2,1425,Aaron Williams,1610612763,VAN,25.0,1996-97,100_poss,33,53.0,0.121,8.0,13.9,0.574,0.0,0.1,0.000,3.1,4.6,0.673,5.8,7.6,13.5,1.4,3.0,1.5,2.7,19.1,-9.8
3,768,Acie Earl,1610612749,MIL,27.0,1996-97,100_poss,47,50.7,0.298,6.8,18.1,0.374,0.0,0.5,0.000,5.5,8.5,0.643,3.5,6.2,9.7,2.0,3.5,1.5,2.8,19.0,-6.1
4,228,Adam Keefe,1610612762,UTA,27.0,1996-97,100_poss,62,51.9,0.774,4.6,9.1,0.513,0.0,0.1,0.000,4.0,5.8,0.689,4.2,8.0,12.2,1.8,2.5,1.7,0.7,13.3,6.9


In [30]:
per_36_basic.head(5)

,player_id,player_name,team_id,team_abbreviation,age,season,mode,gp,min,w_pct,fgm,fga,fg_pct,fg3m,fg3a,fg3_pct,ftm,fta,ft_pct,oreb,dreb,reb,ast,tov,stl,blk,pts,plus_minus
0,920,A.C. Green,1610612742,DAL,33.0,1996-97,36_min,83,2494.298333,0.277,3.4,7.0,0.483,0.0,0.3,0.050,1.8,2.8,0.650,3.2,6.3,9.5,1.0,1.1,1.0,0.2,8.6,-5.1
1,243,Aaron McKie,1610612765,DET,24.0,1996-97,36_min,83,1623.911667,0.578,3.3,8.1,0.411,0.9,2.3,0.398,2.0,2.4,0.836,0.9,4.0,4.9,3.6,2.0,1.7,0.5,9.6,2.9
2,1425,Aaron Williams,1610612763,VAN,25.0,1996-97,36_min,33,562.423333,0.121,5.4,9.5,0.574,0.0,0.1,0.000,2.1,3.1,0.673,4.0,5.2,9.2,1.0,2.0,1.0,1.9,13.0,-6.7
3,768,Acie Earl,1610612749,MIL,27.0,1996-97,36_min,47,500.141667,0.298,4.8,12.9,0.374,0.0,0.4,0.000,3.9,6.0,0.643,2.5,4.4,6.9,1.4,2.5,1.1,2.0,13.5,-4.3
4,228,Adam Keefe,1610612762,UTA,27.0,1996-97,36_min,62,916.788333,0.774,3.2,6.3,0.513,0.0,0.0,0.000,2.8,4.0,0.689,2.9,5.5,8.5,1.3,1.8,1.2,0.5,9.2,4.8


In [31]:
totals_adv.head(5)

,player_id,player_name,team_id,team_abbreviation,season,mode,w_pct,min,off_rating,def_rating,net_rating,ast_to,ast_ratio,oreb_pct,dreb_pct,reb_pct,e_tov_pct,efg_pct,ts_pct,usg_pct,pie
0,920,A.C. Green,1610612742,DAL,1996-97,total,0.277,30.1,97.4,104.8,-7.4,0.93,9.6,0.089,0.184,0.135,10.4,0.485,0.523,0.118,0.098
1,243,Aaron McKie,1610612765,DET,1996-97,total,0.578,19.6,101.5,97.8,3.7,1.79,24.2,0.026,0.113,0.071,13.5,0.467,0.524,0.142,0.095
2,1425,Aaron Williams,1610612763,VAN,1996-97,total,0.121,17.0,93.3,102.6,-9.3,0.47,7.0,0.113,0.144,0.129,14.8,0.574,0.599,0.161,0.112
3,768,Acie Earl,1610612749,MIL,1996-97,total,0.298,10.6,93.6,100.0,-6.4,0.57,7.3,0.067,0.122,0.094,12.9,0.374,0.435,0.220,0.056
4,228,Adam Keefe,1610612762,UTA,1996-97,total,0.774,14.8,106.6,99.4,7.2,0.71,11.3,0.096,0.158,0.129,15.9,0.513,0.572,0.124,0.083


Here we join the basic and advanced metrics tables together into one

In [32]:
def join_basic_adv_tables(df1, df2):
    # Join the tables on these columns
    combined_df = df1.merge(df2, on=['player_id', 'season', 'player_name', 'team_abbreviation', 'team_id'])
    
    # Grab the desired columns and send them into a new dataframe
    # combined_stats = ['player_id', 'player_name', 'team_abbreviation', 'season', 'age', 'w_pct_x',\
    #                   #'min_x', 'min_y',\
    #                   'fgm', 'fga', 'fg_pct', 'ftm', 'fta', 'ft_pct', 'oreb', 'dreb', 'reb',\
    #                   'ast', 'stl', 'blk', 'pts', 'plus_minus', 'off_rating', 'def_rating',\
    #                   'net_rating', 'ast_to', 'ast_ratio', 'oreb_pct', 'dreb_pct', 'reb_pct',\
    #                   'e_tov_pct', 'efg_pct', 'ts_pct', 'usg_pct', 'pie']
    # combined_df_updated = combined_df[combined_stats]
    combined_df['stops'] = combined_df['stl'] + combined_df['blk']
    
    combined_stats = ['player_id', 'player_name', 'team_abbreviation', 'team_id', 'season', 'age',\
                      'fg_pct', 'fg3_pct', 'oreb', 'reb',\
                      'stops', 'stl', 'blk', 'pts', 'plus_minus', 'off_rating', 'def_rating',\
                      'net_rating', 'ast_to', 'oreb_pct',\
                      'e_tov_pct', 'efg_pct', 'ts_pct', 'usg_pct', 'pie']
    combined_df_updated = combined_df[combined_stats]
    return combined_df_updated

def join_rest_tables(df1, df2):
    # Join the tables on these columns
    combined_df = df1.merge(df2, on=['player_id', 'season', 'player_name', 'team_abbreviation', 'team_id'])
    
    combined_stats = ['player_id', 'player_name', 'team_abbreviation', 'season', 'age',\
                      'fg2_pct', 'fg3_pct', 'stops', 'pts', 'off_rating', 'def_rating',\
                      'ast_to', 'oreb_pct', 'e_tov_pct',\
                      'efg_pct', 'ts_pct', 'usg_pct', 'pie'\
                     # ,'fg2a_frequency', 'fg3a_frequency', 'fg2a', 'fg3a'
                     ]
    
    combined_df_updated = combined_df[combined_stats]
    return combined_df_updated
    
# Rename the columns
def rename_columns(df):
    df_renamed = df.rename(columns={'team_abbreviation':'team_abbr',\
                              'off_rating': 'ortg',\
                              'def_rating': 'drtg',\
                              'net_rating': 'nrtg',\
                              'e_tov_pct': 'tov_pct',\
                              'min_x': 'total_min',\
                              'min_y': 'min_per',\
                             'usg_pct': 'usage',\
                             'w_pct_x': 'w_pct'})
    return df_renamed


Run the per 100 possessions and 36 mins stats here

In [33]:
combined_stats_per_100_updated = join_basic_adv_tables(per_100_basic, totals_adv)
combined_stats_per_36_updated = join_basic_adv_tables(per_36_basic, totals_adv)
combined_all_per_100 = join_rest_tables(combined_stats_per_100_updated, totals_shooting)
combined_all_per_36 = join_rest_tables(combined_stats_per_36_updated, totals_shooting)

combined_stats_per_100_updated = rename_columns(combined_stats_per_100_updated)
combined_stats_per_36_updated = rename_columns(combined_stats_per_36_updated)
combined_all_per_100 = rename_columns(combined_all_per_100)
combined_all_per_36 = rename_columns(combined_all_per_36)

In [34]:
combined_stats_per_100_updated.shape

(13380, 25)

In [35]:
combined_stats_per_36_updated.shape

(13380, 25)

In [36]:
combined_all_per_100.shape

(5716, 18)

In [37]:
combined_all_per_36.shape

(5716, 18)

In [38]:
totals_shooting.shape

(5716, 10)

In [39]:
# # This code checks to see if there was a difference between the combined data and the totals_shooting data. If there is, run it to see if you
# # need to run the API again
# missing = totals_shooting.merge(combined_all_per_36.drop_duplicates(), on=['player_id', 'season'], how='left', indicator=True)
# missing[missing['_merge'] == 'left_only']

### Need to run this data to see if there's anyone that's got less than 500 minutes per season (or 5 minutes per game in current)

In [40]:
totals_basic

,player_id,player_name,team_id,team_abbreviation,age,season,mode,gp,min,w_pct,fgm,fga,fg_pct,fg3m,fg3a,fg3_pct,ftm,fta,ft_pct,oreb,dreb,reb,ast,tov,stl,blk,pts,plus_minus
0,920,A.C. Green,1610612742,DAL,33.0,1996-97,total,83,2494.298333,0.277,234,484,0.483,1,20,0.050,128,197,0.650,222,434,656,69,74,70,16,597,-350
1,243,Aaron McKie,1610612765,DET,24.0,1996-97,total,83,1623.911667,0.578,150,365,0.411,41,103,0.398,92,110,0.836,40,181,221,161,90,77,22,433,133
2,1425,Aaron Williams,1610612763,VAN,25.0,1996-97,total,33,562.423333,0.121,85,148,0.574,0,1,0.000,33,49,0.673,62,81,143,15,32,16,29,203,-104
3,768,Acie Earl,1610612749,MIL,27.0,1996-97,total,47,500.141667,0.298,67,179,0.374,0,5,0.000,54,84,0.643,35,61,96,20,35,15,28,188,-60
4,228,Adam Keefe,1610612762,UTA,27.0,1996-97,total,62,916.788333,0.774,82,160,0.513,0,1,0.000,71,103,0.689,75,141,216,32,45,30,13,235,122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13375,1628380,Zach Collins,1610612759,SAS,26.0,2023-24,total,36,910.973333,0.194,167,349,0.479,36,124,0.290,70,91,0.769,68,136,204,115,80,20,26,440,-249
13376,203897,Zach LaVine,1610612741,CHI,28.0,2023-24,total,25,871.861667,0.400,170,376,0.452,59,169,0.349,88,103,0.854,8,121,129,98,52,21,8,487,-92
13377,1630192,Zeke Nnaji,1610612743,DEN,23.0,2023-24,total,36,338.700000,0.722,41,88,0.466,5,16,0.313,25,39,0.641,39,36,75,19,23,9,20,112,-118
13378,1630533,Ziaire Williams,1610612763,MEM,22.0,2023-24,total,44,828.348333,0.409,120,308,0.390,49,156,0.314,49,59,0.831,30,119,149,64,57,27,9,338,-221


In [41]:
total_mins = totals_basic[['player_id', 'player_name', 'season', 'min', 'gp']].copy()
total_mins['mpg'] = total_mins['min'] / total_mins['gp']

In [42]:
combined_stats_per_36_updated = combined_stats_per_36_updated.merge(total_mins, on=['player_id', 'player_name', 'season'])
combined_stats_per_100_updated = combined_stats_per_100_updated.merge(total_mins, on=['player_id', 'player_name', 'season'])
combined_all_per_36 = combined_all_per_36.merge(total_mins, on=['player_id', 'player_name', 'season'])
combined_all_per_100 = combined_all_per_100.merge(total_mins, on=['player_id', 'player_name', 'season'])

In [43]:
combined_stats_per_36_updated = combined_stats_per_36_updated[combined_stats_per_36_updated['mpg'] >= 5]
combined_stats_per_100_updated = combined_stats_per_100_updated[combined_stats_per_100_updated['mpg'] >= 5]
combined_all_per_36 = combined_all_per_36[combined_all_per_36['mpg'] >= 5]
combined_all_per_100 = combined_all_per_100[combined_all_per_100['mpg'] >= 5]

Drop the min column from the data set

In [44]:
combined_stats_per_36_updated = combined_stats_per_36_updated.drop(columns='min')
combined_stats_per_100_updated = combined_stats_per_100_updated.drop(columns='min')
combined_all_per_36 = combined_all_per_36.drop(columns='min')
combined_all_per_100 = combined_all_per_100.drop(columns='min')

Move the GP column towards the beginning

In [45]:
combined_stats_per_36_updated.insert(5, 'gp', combined_stats_per_36_updated.pop('gp'))
combined_stats_per_100_updated.insert(5, 'gp', combined_stats_per_100_updated.pop('gp'))
combined_all_per_36.insert(5, 'gp', combined_all_per_36.pop('gp'))
combined_all_per_100.insert(5, 'gp', combined_all_per_100.pop('gp'))

Move the MPG column towards the beginning

In [46]:
combined_stats_per_36_updated.insert(6, 'mpg', combined_stats_per_36_updated.pop('mpg'))
combined_stats_per_100_updated.insert(6, 'mpg', combined_stats_per_100_updated.pop('mpg'))
combined_all_per_36.insert(6, 'mpg', combined_all_per_36.pop('mpg'))
combined_all_per_100.insert(6, 'mpg', combined_all_per_100.pop('mpg'))

In [47]:
combined_stats_per_36_updated['mpg'] = round(combined_stats_per_36_updated['mpg'], 2)
combined_stats_per_100_updated['mpg'] = round(combined_stats_per_100_updated['mpg'], 2)
combined_all_per_36['mpg'] = round(combined_all_per_36['mpg'], 2)
combined_all_per_100['mpg'] = round(combined_all_per_100['mpg'], 2)

In [48]:
combined_all_per_100

,player_id,player_name,team_abbr,season,age,gp,mpg,fg2_pct,fg3_pct,stops,pts,ortg,drtg,ast_to,oreb_pct,tov_pct,efg_pct,ts_pct,usage,pie
1,201166,Aaron Brooks,DEN,2013-14,29.0,72,21.63,0.458,0.387,2.0,20.0,104.1,106.7,1.99,0.027,12.0,0.484,0.518,0.201,0.079
2,201189,Aaron Gray,SAC,2013-14,29.0,37,9.58,0.521,0.000,2.5,9.1,97.6,108.7,0.71,0.113,25.2,0.443,0.466,0.122,0.044
3,203519,Adonis Thomas,PHI,2013-14,21.0,6,6.26,0.625,0.200,0.0,19.2,108.2,91.0,3.00,0.000,5.4,0.464,0.485,0.176,0.040
4,1733,Al Harrington,WAS,2013-14,34.0,34,15.04,0.495,0.340,1.4,22.2,106.8,102.1,0.82,0.032,12.0,0.478,0.506,0.225,0.061
5,201143,Al Horford,ATL,2013-14,28.0,29,33.03,0.592,0.364,3.7,27.7,104.3,101.1,1.19,0.072,10.7,0.571,0.588,0.244,0.153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5711,1628380,Zach Collins,SAS,2023-24,26.0,36,25.30,0.582,0.290,2.3,22.6,106.6,118.9,1.44,0.071,13.7,0.530,0.565,0.214,0.087
5712,203897,Zach LaVine,CHI,2023-24,28.0,25,34.87,0.539,0.349,1.6,27.4,109.9,115.3,1.88,0.009,9.1,0.531,0.578,0.238,0.110
5713,1630192,Zeke Nnaji,DEN,2023-24,23.0,36,9.41,0.500,0.313,4.2,16.2,100.4,115.0,0.83,0.106,15.6,0.494,0.533,0.159,0.053
5714,1630533,Ziaire Williams,MEM,2023-24,22.0,44,18.83,0.473,0.314,2.1,19.5,102.6,114.0,1.12,0.033,12.5,0.469,0.506,0.196,0.071


### Finalize the data and put them into CSVs

In [49]:
# Finalize the CSVs
combined_stats_per_36_updated.to_csv('per_36_mins_combined_stats.csv', index=False)
combined_stats_per_100_updated.to_csv('per_100_poss_combined_stats.csv', index=False)
combined_all_per_36.to_csv('per_36_mins_combined_all.csv', index=False)
combined_all_per_100.to_csv('per_100_poss_combined_all.csv', index=False)

### Creating the pivot table

This is to put all the column labels into one column while having the stats in the other. This is important
for creating the radar chart.

In [50]:
# combined_stats_per_36_updated.melt(id_vars=['player_id', 'player_name', 'team_abbr', 'season', 'age', 'w_pct', 'total_min', 'min_per'])
stats_per_36_pivot = combined_all_per_36.melt(id_vars=['player_id', 'player_name', 'team_abbr', 'season', 'age', 'gp', 'mpg'],\
                                                       var_name='stat')
stats_per_100_pivot = combined_all_per_100.melt(id_vars=['player_id', 'player_name', 'team_abbr', 'season', 'age', 'gp', 'mpg'],\
                                                         var_name='stat')

In [51]:
columns = {'fg2_pct': 1, 'fg3_pct': 2, 'efg_pct': 3, 'ts_pct': 4,  'pts': 5,\
           'oreb_pct': 6, 'ast_to': 7,  'tov_pct': 8, 'stops': 9,\
           'usage': 10, 'ortg': 11, 'drtg': 12, 'pie': 13}

stats_per_36_pivot['stat_id'] = stats_per_36_pivot['stat'].map(columns)
stats_per_100_pivot['stat_id'] = stats_per_100_pivot['stat'].map(columns)

In [52]:
mask = (stats_per_100_pivot['player_name'] == 'Aaron Brooks') & (stats_per_100_pivot['season'] == '2013-14')
stats_per_100_pivot[mask]

,player_id,player_name,team_abbr,season,age,gp,mpg,stat,value,stat_id
0,201166,Aaron Brooks,DEN,2013-14,29.0,72,21.63,fg2_pct,0.458,1
5389,201166,Aaron Brooks,DEN,2013-14,29.0,72,21.63,fg3_pct,0.387,2
10778,201166,Aaron Brooks,DEN,2013-14,29.0,72,21.63,stops,2.000,9
16167,201166,Aaron Brooks,DEN,2013-14,29.0,72,21.63,pts,20.000,5
21556,201166,Aaron Brooks,DEN,2013-14,29.0,72,21.63,ortg,104.100,11
26945,201166,Aaron Brooks,DEN,2013-14,29.0,72,21.63,drtg,106.700,12
32334,201166,Aaron Brooks,DEN,2013-14,29.0,72,21.63,ast_to,1.990,7
37723,201166,Aaron Brooks,DEN,2013-14,29.0,72,21.63,oreb_pct,0.027,6
43112,201166,Aaron Brooks,DEN,2013-14,29.0,72,21.63,tov_pct,12.000,8
48501,201166,Aaron Brooks,DEN,2013-14,29.0,72,21.63,efg_pct,0.484,3


In [53]:
percent = ['fg2_pct', 'fg3_pct', 'oreb_pct', 'efg_pct', 'ts_pct', 'usage']

In [54]:
stats_per_36_pivot.to_csv('pivot_stats_per_36.csv', index=False)
stats_per_100_pivot.to_csv('pivot_stats_per_100.csv', index=False)

### Creating percentile ranks

In [55]:
# Create a copy of the dataframes to add the ranking columns
combined_all_per_36_ranks = combined_all_per_36[['player_id', 'player_name', 'team_abbr', 'season', 'age', 'gp', 'mpg']].copy()
combined_all_per_100_ranks = combined_all_per_100[['player_id', 'player_name', 'team_abbr', 'season', 'age', 'gp', 'mpg']].copy()

In [56]:
# This will input the columns from the combined dataframe into the new dataframe but with ranks instead.
# Can make them percentile ranks or something else.
def add_percentile_ranks(df1, df2, column_start=0, type='Max', pct=True):
    for column in df2.columns[column_start : ]:
        if column in ['drtg', 'tov_pct']:
            df1[column] = df2.groupby('season')[column].rank('max', ascending=False, pct=pct)
        else:
            df1[column] = df2.groupby('season')[column].rank('max', pct=pct)
    return df1

In [57]:
combined_all_per_36.columns[5:18]

Index(['gp', 'mpg', 'fg2_pct', 'fg3_pct', 'stops', 'pts', 'ortg', 'drtg',
       'ast_to', 'oreb_pct', 'tov_pct', 'efg_pct', 'ts_pct'],
      dtype='object')

In [58]:
combined_all_per_36_ranks = add_percentile_ranks(combined_all_per_36_ranks, combined_all_per_36, 7)
combined_all_per_100_ranks = add_percentile_ranks(combined_all_per_100_ranks, combined_all_per_100, 7)

In [59]:
ranks_per_36_pivot = combined_all_per_36_ranks.melt(id_vars=['player_id', 'player_name', 'team_abbr', 'season', 'age', 'gp', 'mpg'],\
                                                      var_name='stat', value_name='rank')
ranks_per_100_pivot = combined_all_per_100_ranks.melt(id_vars=['player_id', 'player_name', 'team_abbr', 'season', 'age', 'gp', 'mpg'],\
                                                        var_name='stat', value_name='rank')
ranks_per_36_pivot['rank'] = round(ranks_per_36_pivot['rank'] * 100, 2).astype(int)
ranks_per_100_pivot['rank'] = round(ranks_per_100_pivot['rank'] * 100, 2).astype(int)

columns = {'fg2_pct': 1, 'fg3_pct': 2, 'efg_pct': 3, 'ts_pct': 4,  'pts': 5,\
           'oreb_pct': 6, 'ast_to': 7,  'tov_pct': 8, 'stops': 9,\
           'usage': 12, 'ortg': 11, 'drtg': 10, 'pie': 13}

ranks_per_36_pivot['stat_id'] = ranks_per_36_pivot['stat'].map(columns)
ranks_per_100_pivot['stat_id'] = ranks_per_100_pivot['stat'].map(columns)

In [60]:
ranks_per_36_pivot.sort_values(by=['season', 'player_name'])

,player_id,player_name,team_abbr,season,age,gp,mpg,stat,rank,stat_id
0,201166,Aaron Brooks,DEN,2013-14,29.0,72,21.63,fg2_pct,19,1
5389,201166,Aaron Brooks,DEN,2013-14,29.0,72,21.63,fg3_pct,84,2
10778,201166,Aaron Brooks,DEN,2013-14,29.0,72,21.63,stops,40,9
16167,201166,Aaron Brooks,DEN,2013-14,29.0,72,21.63,pts,64,5
21556,201166,Aaron Brooks,DEN,2013-14,29.0,72,21.63,ortg,58,11
...,...,...,...,...,...,...,...,...,...,...
48500,1629627,Zion Williamson,NOP,2023-24,23.0,38,30.28,tov_pct,36,8
53889,1629627,Zion Williamson,NOP,2023-24,23.0,38,30.28,efg_pct,80,3
59278,1629627,Zion Williamson,NOP,2023-24,23.0,38,30.28,ts_pct,79,4
64667,1629627,Zion Williamson,NOP,2023-24,23.0,38,30.28,usage,93,12


In [61]:
ranks_per_36_pivot.sort_values(by=['season', 'player_name', 'stat_id']).to_csv('pivot_ranks_per_36.csv', index=False)
ranks_per_100_pivot.sort_values(by=['season', 'player_name', 'stat_id']).to_csv('pivot_ranks_per_100.csv', index=False)

In [62]:
a_ranks_per_100 = ranks_per_100_pivot.copy()
b_ranks_per_100 = ranks_per_100_pivot.copy()

In [63]:
a_ranks_per_100['sort'] = 'A'
b_ranks_per_100['sort'] = 'B'
ab_ranks_per_100 = pd.concat([a_ranks_per_100, b_ranks_per_100], ignore_index=True)
ab_ranks_per_100.to_csv('ab_ranks_per_100.csv', index=False)

### Other Analysis

Correlation coefficient between each of the statistics and a player's win %...

In [64]:
# Doing this for the per 36 minutes
for column in combined_stats_per_36_updated.columns[6:]:
    print(f'{column}\t{combined_stats_per_36_updated["w_pct"].corr(combined_stats_per_36_updated[column])}')

KeyError: 'w_pct'

In [ ]:
# Doing it for the per 100 possessions
for column in combined_stats_per_100_updated.columns[6:]:
    print(f'{column}\t{combined_stats_per_100_updated["w_pct"].corr(combined_stats_per_36_updated[column])}')

In [ ]:
for column in combined_stats_per_36_ranks.columns[5:]:
    print(f'{column}\t{combined_stats_per_36_ranks["w_pct"].corr(combined_stats_per_36_ranks[column])}')

In [ ]:
for column in combined_stats_per_100_ranks.columns[5:]:
    print(f'{column}\t{combined_stats_per_100_ranks["w_pct"].corr(combined_stats_per_36_ranks[column])}')

# Other Data Cleaning

Figuring out which players are missing

In [ ]:
players = test[test['min'] >= 500][['player_id', 'season']]

In [ ]:
players.shape

In [ ]:
all_36_players = per_36_min_basic_stats[(per_36_min_basic_stats['min'] >= 500) & (per_36_min_basic_stats['season'] >= '2013-14')][['player_id', 'season']]

In [ ]:
all_36_players.shape

In [ ]:
missing = all_36_players.merge(players.drop_duplicates(), on=['player_id', 'season'], how='left', indicator=True)

In [ ]:
missing[missing['_merge'] == 'left_only']['player_id']

Figuring out players per season

In [ ]:
per_36_min_basic_stats.groupby('season')[['PLAYER_ID']].count()

In [ ]:
mask = per_36_min_basic_stats['season'] == '2012-13'
per_36_min_basic_stats[mask]